This notebook tries to resolve the stackoverflow http://stackoverflow.com/questions/42470137/elasticsearch-how-to-aggregate-and-filter-by-multiple-rows question with an adhoc example

## Init Elasticsearch indexes and data
Lets create the index structure and load data

In [44]:
import json
from elasticsearch import Elasticsearch

#cluster node to query
es = Elasticsearch(['localhost:9200',])
index_name    = 'country_currency'

es.indices.delete(
    index = index_name,
)
es.indices.create(
    index = index_name,
)

records = [
    {'country': 1, 'currency': 1, 'id': 'p01'},
    {'country': 1, 'currency': 2, 'id': 'p02'},
    {'country': 1, 'currency': 3, 'id': 'p03'},
    {'country': 2, 'currency': 1, 'id': 'p04'},
    {'country': 2, 'currency': 2, 'id': 'p05'},
    {'country': 2, 'currency': 4, 'id': 'p06'},
    {'country': 3, 'currency': 1, 'id': 'p07'},
    {'country': 3, 'currency': 3, 'id': 'p08'},
    {'country': 3, 'currency': 4, 'id': 'p09'},
    {'country': 4, 'currency': 5, 'id': 'p10'},
    {'country': 4, 'currency': 6, 'id': 'p11'},
    {'country': 4, 'currency': 7, 'id': 'p12'}
]
    
kwargs = {
    'body' : []
}
index_config = { 
    "_type"  : "item", 
    "_index" : 'country_currency', 
}

for idx,r in enumerate(records):
    _index_config = dict(index_config)
    _index_config['_id'] = idx
    kwargs['body'].append({'index' : _index_config})
    kwargs['body'].append(r)

_ = es.bulk(**kwargs)

In [45]:
records

[{'country': 1, 'currency': 1, 'id': 'p01'},
 {'country': 1, 'currency': 2, 'id': 'p02'},
 {'country': 1, 'currency': 3, 'id': 'p03'},
 {'country': 2, 'currency': 1, 'id': 'p04'},
 {'country': 2, 'currency': 2, 'id': 'p05'},
 {'country': 2, 'currency': 4, 'id': 'p06'},
 {'country': 3, 'currency': 1, 'id': 'p07'},
 {'country': 3, 'currency': 3, 'id': 'p08'},
 {'country': 3, 'currency': 4, 'id': 'p09'},
 {'country': 4, 'currency': 5, 'id': 'p10'},
 {'country': 4, 'currency': 6, 'id': 'p11'},
 {'country': 4, 'currency': 7, 'id': 'p12'}]

In [48]:
currencies_criteria = [1,2]

elasticsearch_query = {
    "query": {
        "filtered": {
            "filter": {
                "bool": {
                    "must" : [
                        {
                            "term": {"currency" : currencies_criteria}
                        }
                    ]
                    
                }
            }
        }
    },
    "aggs" : {
        "country_count" : {
            "terms" : {
                "field" : "country",
                #https://www.elastic.co/guide/en/elasticsearch/reference/current/search-aggregations-bucket-terms-aggregation.html#search-aggregations-bucket-terms-aggregation-approximate-counts
                "size"  : 0
            }
        }
    }
}

es_kwargs = { 
    "doc_type"  : "item", 
    "index" : "country_currency", 
    "body" : elasticsearch_query,
}


res = es.search(**es_kwargs)

for bucket in res.get('aggregations',{}).get('country_count',{}).get('buckets',[]):
    country = bucket['key']
    print country

1
2
